In [1]:
# STT의 공방은 https://github.com/anuragseven/azurespeech-pyqt5/blob/main/speech.py를 참고
from PyQt5 import QtGui, QtCore, QtWidgets

import sys
import ui_main
import numpy as np
import pyqtgraph
import SWHear
import datetime

class ExampleApp(QtWidgets.QMainWindow, ui_main.Ui_MainWindow):
    def __init__(self, parent=None):
        pyqtgraph.setConfigOption('background', 'w') #before loading widget
        super(ExampleApp, self).__init__(parent)
        self.setupUi(self)
        self.grFFT.plotItem.showGrid(True, True, 0.7)
        self.grPCM.plotItem.showGrid(True, True, 0.7)
        self.maxFFT=0
        self.maxPCM=0
        self.ear = SWHear.SWHear(rate=44100,updatesPerSecond=20)
        self.ear.stream_start()

        
    def on_speech_recognized(self, text):
        self.start_listening.setDisabled(False)
        self.recognised_text_box.setText(text)
        
    def update(self):
        if not self.ear.data is None and not self.ear.fft is None:
            pcmMax=np.max(np.abs(self.ear.data))
            if pcmMax>self.maxPCM:
                self.maxPCM=pcmMax
                self.grPCM.plotItem.setRange(yRange=[-pcmMax,pcmMax])
            if np.max(self.ear.fft)>self.maxFFT:
                self.maxFFT=np.max(np.abs(self.ear.fft))
                #self.grFFT.plotItem.setRange(yRange=[0,self.maxFFT])
                self.grFFT.plotItem.setRange(yRange=[0,1])
            self.pbLevel.setValue(1000*pcmMax/self.maxPCM)
            pen=pyqtgraph.mkPen(color='b')
            self.grPCM.plot(self.ear.datax ,self.ear.data ,pen=pen ,clear=True)
            pen=pyqtgraph.mkPen(color='r')
            self.grFFT.plot(self.ear.fftx, self.ear.fft/self.maxFFT ,pen=pen ,clear=True)
            
        # 라벨에 텍스트 표시
        current_time = str(datetime.datetime.now().time())
        self.label_stt.setText(current_time)
        self.label_stt.repaint()
        QtCore.QTimer.singleShot(1, self.update) # QUICKLY repeat

        


if __name__=="__main__":
    app = QtWidgets.QApplication(sys.argv)
    form = ExampleApp()
    form.show()
    form.update() #start with something
    app.exec_()
    print("DONE")

found 5 microphone devices: [0, 1, 5, 6, 12]
recording from "Microsoft 사운드 매퍼 - Input" (device 0) at 44100 Hz
 -- starting stream
DONE
